# Day 22

## Part 1

Enough sand has fallen; it can finally filter water for Snow Island.

Well, almost.

The sand has been falling as large compacted bricks of sand, piling up to form an impressive stack here near the edge of Island Island. In order to make use of the sand to filter water, some of the bricks will need to be broken apart - nay, disintegrated - back into freely flowing sand.

The stack is tall enough that you'll have to be careful about choosing which bricks to disintegrate; if you disintegrate the wrong brick, large portions of the stack could topple, which sounds pretty dangerous.

The Elves responsible for water filtering operations took a snapshot of the bricks while they were still falling (your puzzle input) which should let you work out which bricks are safe to disintegrate. For example:

```
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
```
Each line of text in the snapshot represents the position of a single brick at the time the snapshot was taken. The position is given as two x,y,z coordinates - one for each end of the brick - separated by a tilde (~). Each brick is made up of a single straight line of cubes, and the Elves were even careful to choose a time for the snapshot that had all of the free-falling bricks at integer positions above the ground, so the whole snapshot is aligned to a three-dimensional cube grid.

A line like 2,2,2~2,2,2 means that both ends of the brick are at the same coordinate - in other words, that the brick is a single cube.

Lines like 0,0,10~1,0,10 or 0,0,10~0,1,10 both represent bricks that are two cubes in volume, both oriented horizontally. The first brick extends in the x direction, while the second brick extends in the y direction.

A line like 0,0,1~0,0,10 represents a ten-cube brick which is oriented vertically. One end of the brick is the cube located at 0,0,1, while the other end of the brick is located directly above it at 0,0,10.

The ground is at z=0 and is perfectly flat; the lowest z value a brick can have is therefore 1. So, 5,5,1~5,6,1 and 0,2,1~0,2,5 are both resting on the ground, but 3,3,2~3,3,3 was above the ground at the time of the snapshot.

Because the snapshot was taken while the bricks were still falling, some bricks will still be in the air; you'll need to start by figuring out where they will end up. Bricks are magically stabilized, so they never rotate, even in weird situations like where a long horizontal brick is only supported on one end. Two bricks cannot occupy the same position, so a falling brick will come to rest upon the first other brick it encounters.

Here is the same example again, this time with each brick given a letter so it can be marked in diagrams:

```
1,0,1~1,2,1   <- A
0,0,2~2,0,2   <- B
0,2,3~2,2,3   <- C
0,0,4~0,2,4   <- D
2,0,5~2,2,5   <- E
0,1,6~2,1,6   <- F
1,1,8~1,1,9   <- G
```
At the time of the snapshot, from the side so the x axis goes left to right, these bricks are arranged like this:

```
 x
012
.G. 9
.G. 8
... 7
FFF 6
..E 5 z
D.. 4
CCC 3
BBB 2
.A. 1
--- 0
```
Rotating the perspective 90 degrees so the y axis now goes left to right, the same bricks are arranged like this:

```
 y
012
.G. 9
.G. 8
... 7
.F. 6
EEE 5 z
DDD 4
..C 3
B.. 2
AAA 1
--- 0
```
Once all of the bricks fall downward as far as they can go, the stack looks like this, where ? means bricks are hidden behind other bricks at that location:

```
 x
012
.G. 6
.G. 5
FFF 4
D.E 3 z
??? 2
.A. 1
--- 0
```
Again from the side:

```
 y
012
.G. 6
.G. 5
.F. 4
??? 3 z
B.C 2
AAA 1
--- 0
```
Now that all of the bricks have settled, it becomes easier to tell which bricks are supporting which other bricks:

- rick A is the only brick supporting bricks B and C.
- Brick B is one of two bricks supporting brick D and brick E.
- Brick C is the other brick supporting brick D and brick E.
- Brick D supports brick F.
- Brick E also supports brick F.
- Brick F supports brick G.
- Brick G isn't supporting any bricks.
Your first task is to figure out which bricks are safe to disintegrate. A brick can be safely disintegrated if, after removing it, no other bricks would fall further directly downward. Don't actually disintegrate any bricks - just determine what would happen if, for each brick, only that brick were disintegrated. Bricks can be disintegrated even if they're completely surrounded by other bricks; you can squeeze between bricks if you need to.

In this example, the bricks can be disintegrated as follows:

- Brick A cannot be disintegrated safely; if it were disintegrated, bricks B and C would both fall.
- Brick B can be disintegrated; the bricks above it (D and E) would still be supported by brick C.
- Brick C can be disintegrated; the bricks above it (D and E) would still be supported by brick B.
- Brick D can be disintegrated; the brick above it (F) would still be supported by brick E.
- Brick E can be disintegrated; the brick above it (F) would still be supported by brick D.
- Brick F cannot be disintegrated; the brick above it (G) would fall.
- Brick G can be disintegrated; it does not support any other bricks.
So, in this example, 5 bricks can be safely disintegrated.

Figure how the blocks will settle based on the snapshot. Once they've settled, consider disintegrating a single brick; how many bricks could be safely chosen as the one to get disintegrated?

In [3]:
from shapely.geometry import Polygon, MultiPolygon
import plotly.graph_objects as go
import string
from rich import print as rprint


class Brick3D:
    offset = 0.5

    def __init__(self, coords1: str, coords2: str):
        x1, y1, z1 = [int(val) for val in coords1.split(",")]
        x2, y2, z2 = [int(val) for val in coords2.split(",")]

        self.xmin = min(x1, x2) - self.offset
        self.xmax = max(x1, x2) + self.offset

        self.ymin = min(y1, y2) - self.offset
        self.ymax = max(y1, y2) + self.offset

        self.zmin = min(z1, z2) - self.offset
        self.zmax = max(z1, z2) + self.offset

    def drop(self, amount: int = 1) -> None:
        self.zmin -= amount
        self.zmax -= amount

    @property
    def zface(self) -> Polygon:
        # represents the top/bottom face of the brick, which will have the same z value;
        # used for checking if a top/bottom face overlaps with another brick's bottom/top face
        return Polygon(
            [
                (self.xmin, self.ymin),
                (self.xmin, self.ymax),
                (self.xmax, self.ymax),
                (self.xmax, self.ymin),
                (self.xmin, self.ymin),  # close the loop
            ]
        )

    def has_polygon_intersection_with(self, other: "Brick3D") -> bool:
        if not self.zface.intersects(other.zface):
            return False
        intersection = self.zface.intersection(other.zface)
        # any polygons that are right next to each other will have a LineString intersection, and
        # we don't care about that case here -- just polygon intersections
        return isinstance(intersection, Polygon) and not intersection.is_empty

    def is_supported_by(self, other: "Brick3D") -> bool:
        return self.zmin == other.zmax and self.has_polygon_intersection_with(other)

    def is_supporting(self, other: "Brick3D") -> bool:
        return self.zmax == other.zmin and self.has_polygon_intersection_with(other)

    @property
    def is_touching_ground(self) -> bool:
        return self.zmin <= 1

    def __repr__(self) -> str:
        return f"<Brick3D x=[{self.xmin},{self.xmax}], y=[{self.ymin},{self.ymax}], z=[{self.zmin}, {self.zmax}]>"


class BrickDropper:
    def __init__(self, bricks: list[Brick3D]):
        self.bricks: list[Brick3D] = sorted(bricks, key=lambda brick: brick.zmin)

    def settle(self, brick: Brick3D) -> None:
        new_brick_order = []

        for i, brick in enumerate(self.bricks):
            # print(f"{i}: {brick.is_touching_ground=}")
            while not brick.is_touching_ground:
                if any(brick.is_supported_by(other) for other in self.bricks):
                    # print(f"\t{i}: supported by another brick, not dropping")
                    break
                # print(f"\t{i}: dropping brick")
                brick.drop()
                # print(f"\t{i}: z={brick.zmin + brick.offset}, {brick.is_touching_ground=}")
            new_brick_order.append(brick)

        self.bricks = sorted(new_brick_order, key=lambda brick: brick.zmin)

    def count_disintegratable_bricks(self) -> list[Brick3D]:
        disintegratable_bricks = set()

        for brick in sorted(self.bricks, key=lambda brick: brick.zmin):
            # first, check if this brick is supporting any others
            other_bricks_supported = [other for other in self.bricks if brick.is_supporting(other)]
            if not other_bricks_supported:
                # this can be disintegrated without any others dropping
                disintegratable_bricks.add(brick)
                continue

            # if it *is* supporting other bricks, we need to see if there are any other supporting
            # bricks that would take over if this one were to be disintegrated
            for supported_brick in other_bricks_supported:
                other_supporting_bricks = [
                    other_brick
                    for other_brick in self.bricks
                    if other_brick != brick and other_brick.is_supporting(supported_brick)
                ]
                if other_supporting_bricks:
                    disintegratable_bricks.add(brick)
                    break

        return len(disintegratable_bricks)


def make_bricks(inputs: list[str]):
    bricks = []
    for line in inputs:
        coords1, coords2 = line.split("~")
        brick = Brick3D(coords1, coords2)
        bricks.append(brick)
    return bricks


def plot_bricks(bricks: list[Brick3D], use_alpha: bool = False) -> None:
    fig = go.Figure()

    for i, brick in enumerate(bricks):
        vertices = [
            (brick.xmin, brick.ymin, brick.zmin),
            (brick.xmax, brick.ymin, brick.zmin),
            (brick.xmax, brick.ymax, brick.zmin),
            (brick.xmin, brick.ymax, brick.zmin),
            (brick.xmin, brick.ymin, brick.zmax),
            (brick.xmax, brick.ymin, brick.zmax),
            (brick.xmax, brick.ymax, brick.zmax),
            (brick.xmin, brick.ymax, brick.zmax),
        ]
        edges_order = [0, 1, 2, 3, 0, 4, 5, 6, 7, 4, 5, 1, 6, 2, 7, 3]
        x_coords = [vertices[i][0] for i in edges_order]
        y_coords = [vertices[i][1] for i in edges_order]
        z_coords = [vertices[i][2] for i in edges_order]

        name = str(i)
        if use_alpha:
            name = str(string.ascii_uppercase[i])

        fig.add_trace(
            go.Scatter3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                mode="lines",
                name=name,
            )
        )

    fig.update_layout(
        template="plotly_dark",
        scene=dict(
            xaxis=dict(
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
            yaxis=dict(
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
            zaxis=dict(
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
        ),
        height=600,
        width=400,
        margin=dict(r=5, l=5, b=5, t=5),
    )
    fig.show()


test_input = """
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9""".strip().splitlines()

test_bricks = make_bricks(test_input)
test_dropper = BrickDropper(test_bricks)

plot_bricks(test_dropper.bricks)  # before
test_dropper.settle(test_bricks)
plot_bricks(test_dropper.bricks)  # after

test_count = test_dropper.count_disintegratable_bricks()
print(test_count)
assert test_count == 5, test_count

5


In [4]:
inputs = open("../inputs/22.txt").read().strip().splitlines()
bricks = make_bricks(inputs)
print(len(bricks))
dropper = BrickDropper(bricks)
plot_bricks(dropper.bricks[:200])

1394


In [7]:
dropper.settle(bricks)
plot_bricks(dropper.bricks[:300])
count = dropper.count_disintegratable_bricks()
count

549